In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Qiskit imports
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.circuit.library import zz_feature_map, real_amplitudes
from qiskit_ibm_runtime import EstimatorV2 as Estimator

%load_ext autoreload
%autoreload 2

## Import Data

In [ ]:
# Load data with pandas
data = pd.read_csv("diabetes_normalized.csv")
data.head()

### Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x, y = 
nfeatures = 

In [ ]:
x_train, x_test, y_train, y_test = 
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

## Building the Variational Quantum Classifier (VQC) for Binary Classification Problem

This will include multiple steps, starting with 

A. Building the VQC algorithm 
1. Data Embedding, followed by,
2. Parameterized Quantum Circuit (PQC)/Ansatz with trainable parameters.

B. Optimizing the circuit and operators
1. Setting up Qiskit Runtime Service
2. Traspilation

C. Execution and Results
1. Setup Primitives 

D. Post-processing
1. Visualizing the results

### Step A1: Data Embedding

In [ ]:
from utils import angle_embedding

In [ ]:
feature_params = 
embedding = 
embedding.draw("mpl")

### Step A2: PQC with Trainable Parameters

In [ ]:
ansatz = 
ansatz.draw("mpl")

In [ ]:
weight_params = 
print(weight_params)

### Final VQC Circuit

In [ ]:
vqc_circuit = 
vqc_circuit.draw("mpl")

In [ ]:
# Setting up circuit observables
from qiskit.quantum_info import SparsePauliOp

observable = 

## Step B: Optimize circuits and operators

### Step B1: Setup Qiskit Runtime Service

In [ ]:
# Setup Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.fake_provider import FakeQuebec

# If setting up Qiskit Runtime for the first time with custom hub/group/instance
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="token", instance="hub/group/project", name="CustomName")
# service = QiskitRuntimeService(instance="hub/group/project")

service = QiskitRuntimeService(instance="ibm-q/open/main")
# backend = service.least_busy(simulator=False, operational=True)
fake_backend = FakeQuebec()

### Step B2: Transpilation

In [ ]:
# Build an ISA circuit using pre-set transpilation passes.
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

pm =  generate_preset_pass_manager()   # You can change the optimization level

# To see how the transpiled circuit looks, uncomment the lines below.
# vqc_transpiled_circuit = pm.run(vqc_circuit)
# vqc_transpiled_circuit.draw("mpl", idle_wires=False)

In [ ]:
# Apply obervables to the transpiled layout
mapped_observables = [obs.apply_layout(vqc_transpiled_circuit.layout) for obs in observable]
print(mapped_observables)

## Step C: Execution and Results

### Step C1: Setup Primitives

In [ ]:
# Imports
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_machine_learning.optimizers import COBYLA

In [ ]:
# Estimator Primitive
estimator = 

# Define the QNN Primitive
estimator_qnn = EstimatorQNN(
    estimator= ,
    circuit= ,
    observables= ,
    input_params= ,
    weight_params= ,
    pass_manager= ,
)

In [ ]:
from IPython.display import clear_output

# Callback function
def callback_graph(weights, objective_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(objective_func_eval)
    print(f"COBYLA Epoch {len(objective_func_vals)}: {objective_func_eval:.5f}")
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [ ]:
VQC = NeuralNetworkClassifier(
    neural_network = , 
    optimizer= , 
    callback= ,
)

In [ ]:
objective_func_vals = []

# Fit VQC
VQC

In [ ]:
# Train Score
score_train = VQC.score(x_train, y_train)
print(f'Score on the train set {score_train}')

# Test Score
score_test = VQC.score(x_test, y_test)
print(f'Score on the test set {score_test}')

## Step D: Post-processing and Visualizing the results

In [ ]:
y_test_pred = VQC.predict(x_test)

# Add a column for Test_Label
pairplot_data['Test_Label'] = y_test
# Predicted Label
pairplot_data["Predicted_Label"] = y_test_pred

# print(pairplot_data.head())

In [ ]:
plt.figure(figsize=(10, 8))
scatter = plt.scatter(pairplot_data['Glucose'], pairplot_data['BloodPressure'], c=pairplot_data['Test_Label'], marker='o', edgecolors='k')

# Highlight misclassifications with a different marker
misclassified = pairplot_data[pairplot_data['Test_Label'] != pairplot_data['Predicted_Label']]
plt.scatter(misclassified['Glucose'], misclassified['BloodPressure'], marker='x', s=100, color='red', label='Misclassified')

# Customize the plot
plt.xlabel('Glucose')
plt.ylabel('BloodPressure')
plt.title('Scatter Plot of Features with Test Labels and Predictions')
plt.legend()
plt.colorbar(scatter, label='True Label')

# Show the plot
plt.show()